In [9]:
import pandas as pd
import glob

In [10]:
# Get drug data (NB some drugs duplicated for illnesses)
drug_data = pd.read_csv("drug_list.csv")

In [11]:
# Function to find loneliness related prescribing
def code_loneliness(x):
    out = {}
    # coding by illness categories
    for illness in drug_data['illness'].unique():
        out[illness] = x['BNF_DESCRIPTION'].str.contains("|".join(drug_data[drug_data['illness'] == illness]['medication']),
                                                  case=False, 
                                                  regex=True).astype('int16')
    # Make dataframe
    out = pd.DataFrame(out)
    # Add loneliness related disease binary - avoids double counting some drugs.
    out['loneliness'] = x['BNF_DESCRIPTION'].str.contains("|".join(drug_data['medication'].unique()),
                                                   case = False, 
                                                   regex = True).astype('int16')
    # Return dataframe multiplied by counts of items.
    return out.multiply(x['ITEMS'], axis=0)

In [12]:
def england_data(year):
    path = '../testing/England_data/EPD_'+year+'*.zip'
    big_data = pd.DataFrame()
    for file in glob.glob(path):
        print(file)
        prescribe = pd.read_csv(file, compression='zip', header=0, sep=',', quotechar='"')
        prescribe = prescribe.rename(columns={'YEAR_MONTH':'Date','PRACTICE_NAME':'CenterName','PRACTICE_CODE':'PracCode'})
        loneliness_prescribing = code_loneliness(prescribe[['BNF_DESCRIPTION','ITEMS']])
        prescribe = prescribe.merge(loneliness_prescribing, left_index=True, right_index=True)
        prescribe['pcstrip'] = prescribe['POSTCODE'].str.replace("\s","")
        output = prescribe[['PracCode','depression', 'alzheimers', 'blood pressure',
       'hypertension', 'diabeties', 'cardiovascular disease', 'insomnia',
       'addiction', 'social anxiety', 'ITEMS', 'loneliness','Date','pcstrip',
       'ADDRESS_1', 'ADDRESS_2', 'ADDRESS_3', 'ADDRESS_4','POSTCODE']]
        big_data = pd.concat((big_data,output),ignore_index=True,axis=0)
    # big_data.to_csv(year+'England_data.csv')
    return big_data

In [13]:
years = ['2018','2020','2022']

In [14]:
a = england_data('2020')

In [15]:
print(a)

Empty DataFrame
Columns: []
Index: []
